In [1]:
#hide
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
!pip install voila
!jupyter serverextension enable --sys-prefix voila 

## Nosso primeiro classificador de Ursos (em breve serão os peixes) =)

Nesse primeiro teste de Machine Learning, conseguimos realizar um classificador de ursos (pretos, marrons e os de pelúcia). Usando um modelo de Resnet18, nossa máquina de detecção funciona apenas com essas três categorias de ursos. 
Em breve, nossos ursos serão a inspiração para realizar um outro desafio, a identificação dos melhores peixes da Região Norte (de acordo com gosto do Professor Clay =)).

##English
In this first test of Machine Learning, we managed to perform a bear classifier (black, brown and teddy). Using a model of Resnet18, for now, our detection machine works only with these three categories of bears.
Soon, our bears will be the inspiration to carry out another challenge, the identification of the best fish in the North Region (according to Professor Clay's taste =)).

##Français
Dans ce premier test de Machine Learning, nous avons réussi à effectuer un classificateur d'ours (noir, marron et nounours). En utilisant un modèle de Resnet18, pour l'instant, notre machine de détection ne fonctionne qu'avec ces trois catégories d'ours.
Bientôt, nos ours seront l'inspiration pour relever un autre défi, l'identification des meilleurs poissons de la région Nord (selon le goût du professeur Clay =)).

In [3]:
path = Path()
learn_inf = load_learner(path/'export.pkl', cpu=True)
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [9]:
# test def on_click_classify(change):
def on_click(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

#btn_run.on_click(on_click_classify)

In [6]:
#hide
#Putting back btn_upload to a widget for next cell
# test btn_upload = widgets.FileUpload()

In [10]:
btn_upload.observe(on_click, names=['data'])

In [12]:
#hide_output
display(VBox([widgets.Label('Selecione seu urso!'), 
      btn_upload, out_pl, lbl_pred]))
#but_run